# Retrying to Pac-man

follow up work from:

## TP Final - Procesos Markovianos para el Aprendizaje Automatico - 2019 1C

https://github.com/LecJackS/TP-Final-Procesos-Markovianos-para-el-Aprendizaje-Automatico-2019-1C-w.o.heavy-history-

En las pasadas 4 semanas, luego de la entrega, seguí realizando pruebas para asegurarme que el algoritmo funcione (algo que **no** fue claramente demostrado en la entrega anterior).

Para eso se utilizó  [un nuevo environment de pacman (mspacman)](https://gym.openai.com/envs/MsPacman-v0/) que permitiera realizar experimentos a mayor velocidad.

![mspacman.png](./img/mspacman.png)

De esta forma se pudieron realizar ajustes que **demostraron buenos resultados** en el juego, **generalizando también** al lento pacman de Berkeley.

A continuación, una lista de correcciones y resultados obtenidos a partir de este nuevo entorno.

# Statistics

Una **prioridad** para esta serie de experimentos fue tener un **mayor grado de información** en cuanto a los resultados de distintos episodios en el tiempo.

Para eso se agregaron las siguientes estadísticas:

* **Promedio (mean)** de rewards acumulados en **últimos 100 episodios**
* **Mediana** del reward acumulado en los **últimos 100 episodios**
* **Desviación estándar** de rewards acumulados en últimos **100 episodios**
* ***"Mediana-estándar"***, como la mediana menos la media sobre desviación estándar $\frac{\text{median}-\text{mean}}{\sigma}$.
  
  A modo de combinar en un mismo plot los rewards promedios por cantidad (mediana) y por valor (mean).
  

### Experimento "*learning-rate*"

Se compararon 3 valores de step size diferentes:

* lr: $10^{-3}$ - ***Rojo***
* lr: $10^{-4}$ - ***Naranja***
* lr: $10^{-5}$ - ***Azul***

![comp-lr.png](./img/comp-lr.png)

![comp-lr-smooth.png](./img/comp-lr-smooth.png)

Ésto dió bastante certeza en que usar un lr de $10^{-4}$ como se venía haciendo, era lo indicado.

Búsquedas más finas no mostraron grandes diferencias, aunque fueron pruebas no tan extensivas en el tiempo.

### Experimento "*número de procesos*"

Un hyperparámetro que agrega A3C es la **cantidad de procesos asincrónicos** a ejecutar.

Se comparó **1 vs 6 procesos durante varios días**.

Los resultados fueron en favor de una mayor cantidad de procesos.

![comp-num-procesos.png](./img/comp-num-procesos.png)
(azul: 6; gris: 1)

# Rewards and *Expected Reward Count*

Como Vlad Mnih menciona en el siguiente video, simplificar los rewards *recortándolos* en valores estándar como -1,0,1 **facilita** la solución del problema para el agente en muchos casos, ya que **convierte el objetivo en obtener más cantidad de veces** rewards, distinto a *mayor reward*, donde existen distintos valores posible para el reward (ej. pacman: +500, +10, +1, 0, -1, -500), y sus combinaciones al calcular $G_t$ puede dar valores parecidos, a partir de combinaciones muy diferentes.

In [3]:
from IPython.display import HTML
# Youtube
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/-mhBD8Frkc4?start=5520" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')
#@1:32:00 https://youtu.be/-mhBD8Frkc4?t=5520

En otras palabras, se **maximiza la esperanza de la *cantidad* de rewards recibidos** (*expected reward count*).

|   +500  |  +10   |  0 |  -1 |    -500|
|------|------|------|------|------|
|  $\downarrow$  |  $\downarrow$  |  $\downarrow$  |  $\downarrow$  |  $\downarrow$  |
|   +1  | +1 |   0  |0|   0  | 0 | 

Solo esta modificación en el código produjo rápidos resultados, lo cuál tiene sentido para pacman, donde cada pellet y ganar el juego es ahora un +1, y el maximo puntaje será comer todas las pellets (y los fantasmas si *scared*).

![pacman-episode-1.gif](./img/pacman-episode-1.gif)

A pesar de haber aprendido bastante bien a recoger las pellets de un mapa de 14x14, demuestra poca cautela con los fantasmas, lo que indicó que podría ser conveniente agregar un valor negativo para el evento en el que hace contacto, y pierde.

|   +500  |  +10   |  0 |  -1 |    -500|
|------|------|------|------|------|
|  $\downarrow$  |  $\downarrow$  |  $\downarrow$  |  $\downarrow$  |  $\downarrow$  |
|   +1  | +1 |   0  |0|   0  | -1 | 

Con este modelado de rewards se obtuvieron los resultados deseados:

* Pac-man comenzó a uir de los fantasmas y a ir más directamente a comer cada pellet (incentivado al terminar el episodio sin completarlo).

Los descuentos de $-1$ por time step fueron ignorados, por devolver resultados positivos pero poco interpretables. 

### Sobre el código

Gym contiene un conveniente paquete de *Wrappers* que permiten envolver al *environment* luego de creado con *gym.make('MsPacman-v0')* modificando funciones como *environment.step(action)* para que devuelva un reward modificado desde cualquier otro lado que sea llamada: 

>    class UnitReward_cs188x(Wrapper):
>        def __init__(self, env=None, monitor=None):
>            super(UnitReward_cs188x, self).__init__(env)
>            
>        def step(self, action):
>            state, reward, done, info = self.env.step(action)
>            if reward > 0:
>                reward = 1.
>            else:
>                reward = 0.
>
>            return state, reward, done, info



# Optimizer

Se implementó ***RMSProp con parámetros compartidos*** para contrastar con los de ***Adam***, previamente utilizado.

No se observó ninguna diferencia en largos experimentos con los mismos hyperparámetros.

# Entrada

Se modificó el tipo de entrada a la red neuronal.

* Manteniendo las **4 rotaciones** de 90°
* Asignando a cada rotación, los valores (intensidades) de su **canal de color correspondiente (RGB)**, y *grayscale* para la última imagen.

![pacman-episode-4.gif](./img/pacman-episode-4.gif)

Originalmente se pensó como alternativa a buscar manualmenmte el mejor equilibrio de grises para la red (variando mean, varianza, etc).

Las primeras pruebas fueron usando ***ms-pacman de Atari***, sin las rotaciones (ya que tiene información extra en la parte inferior de la pantalla) pero usando el mismo tipo de división de colores:

![mspacman-episode-4.gif](./img/mspacman-episode-4.gif)

Partiendo de **pesos ya entrenados** solo con escala de grises, el agente mostró una rápida adaptación al nuevo tipo de entrada:

![learning-after-colours.png](./img/learning-after-colours.png)

# Skip Frame

Otra característica mencionada pero poco explorada en el anterior proyecto, fue el uso de ***skip frame***.

La primera impresión al ejecutar ms-pacman con el mismo modelo anterior, fue de que **la simulación también era lenta**.

El motivo fue el **alto frame rate**, con el cual cada frame era un estado que debía procesarse por la NN.

Para eso se utiliza ***skip frame*** que elije, combina y/o saltea frames consecutivos.

Para DQN, la entrada a la red neuronal fueron 4 frames apilados:

    [max(T-1,  T),
     max(T+3,  T+4),
     max(T+7,  T+8),
     max(T+11, T+12)]
     
Donde `max(T-1, T)` es el máximo *elementwise* entre las matrices de píxeles de **dos frames consecutivos**.

**Descartando otros 2** frames luego de cada operación max().


Para pacman de Atari, se utilizó un filtro similar al de DQN, con una diferencia:

* con 1 solo *skip frame*

        [max(T,    T+1),
         max(T+3,  T+4),
         max(T+6,  T+7),
         max(T+9,  T+10)]

* Sin *skip frames*, solo máximo entre cada dos

        [max(T,    T+1),
         max(T+2,  T+3),
         max(T+4,  T+5),
         max(T+6,  T+7)]

La siguiente imagen muestra la entrada de píxeles de los 4 estados a la red neuronal, calculando máximo entre cada dos, y salteando 1 frame.

![dqn-skip-frame.png](./img/dqn-skip-frame.png)

Se determinó que **el frame descartado limitaba la capacidad de reacción del agente**, por lo que se decidió por no usar skip frames.

El cambio de entrada produjo resultados negativos (esperados) al agente **ya entrenado** con la entrada anterior.

En unas horas de entrenamiento volvió a los valores anteriores y los siguió superando.

# LSTM + Reciclaje + *Finetunning*

Una vez entrenado el agente de modelo neuronal **lineal**, se decidió probar nuevamente una capa LSTM en su lugar.

Para ello:

1. Se **cargaron los pesos ya entrenados** en el modelo **lineal**
2. **Se *"congelaron"* las capas convolucionales** para mantener los filtros/features aprendidos
3. **Se entrenó el resto** del modelo neuronal **hasta ver mejoras** en el resultado
4. Se descongelaron todas las capas y siguió normalmente con el entrenamiento en conjunto

A continuación, el nuevo modelo neuronal con LSTM y las capas convolucionales congeladas siendo entrenado, partiendo de los pesos del modelo lineal:

![lstm-learn.png](./img/lstm-learn.png)

Last but not least
# Clamp Critic (Value Function)

En la sección ["Estrategias de modelado"](https://github.com/LecJackS/TP-Final-Procesos-Markovianos-para-el-Aprendizaje-Automatico-2019-1C-w.o.heavy-history-/blob/master/Very%20quick%20roadmap%20to%20Asynchronous%20Advantage%20Actor%20Critic.ipynb#Estrategias-de-modelado) del trabajo anterior, se notaron **picos muy grandes** en el error (*loss*) calculado, por lo que se optó por poner topes en el mismo de $\frac{1}{\text{step-size}}$, sin conocer el motivo de ellos.

El problema real fue que la **Value Function (*Critic*)** era una salida en la red neuronal que entregaba valores reales, sin cotas de ningún tipo.

Recordando el modelo:

![actor-critic-model.png](./img/actor-critic-model.png)

Dado que el error total se calcula como:

>    total_loss = -actor_loss + critic_loss - opt.beta * entropy_loss

Donde *critic_loss* es:

>    critic_loss = critic_loss + ((Gt - value)\*\*2) / 2

Esa diferencia al cuadrado era la causante de tales magnitudes.

Una solución encontrada, fue **acotar los valores de la value function** entre $\left[-20,\, 20\right]$, siendo los valores que serán comparados con el $G_t$ del episodio en cada comparación (cada step guardado).

La magnitud $20$ surge de ver los plots de rewards promedio alcanzando valores entre 20 y 30 casi ganando el episodio.

Esta magnitud **será particular de cada juego**, como también **de cada instancia en el tiempo**, distintas para un agente entrenado como para uno random.

## Cotas dinámicas

Una mejor solución fue determinar la magnitud de la cota, como el máximo/mínimo Gt obtenido hasta el momento.

> **Inicia** con **cotas para el Crítico** de $[-1, \,1]$.
>
> Al finalizar cada episodio ($n$ steps), **se calcula Gt** de los datos guardados
>
> $G_t^{(n)} = R_{t+1} + \gamma\, R_{t+2}\, + \gamma^2\, R_{t+3} + \, ... \, + \, \gamma^{n-1} R_{t+n} + \gamma^{n}\, \tilde{V}(S_{t+n})$
>
>     1. Si (Gt > cotaMax): 
>
>        cotaMax = Gt
>
>     2. Si (Gt < cotaMin): 
>
>        cotaMin = Gt

Luego:

>      logits, value = local_model(state)
>
>      value.clamp(cotaMin, cotaMax)

De esta forma, los valores máximo/mínimo del Crítico aumentarán **a medida que se vean puntajes más altos**.

### Mejor estimador: *Exponential Cummulative Average*

Calcular valores extremos puede ser **contraproducente** si se observan retornos $G_t$ altos en **instancias iniciales**, donde **la Value Function tiene mucha varianza**.

Un **mejor estimador** para las cotas es un **promedio incremental**, que dé valores dependientes de su repetición en el tiempo:

$$m_{n+1} = m_n + \frac{G_{t+1} - m_n}{n+1}$$

con **pérdida de memoria** (decaimiento exponencial) para valores antiguos:

$$m_{n+1} = \alpha\, m_n + (1-\alpha) \frac{G_{t+1} - m_n}{n+1} \,\, , \, \alpha \in [0,1]$$


Se eligió un $\alpha = 0.495$





De esta manera, **en cada decisión** del agente, habrá una **estimación del valor del siguiente estado**, siendo un valor entre $[-1,\,1]$, y un consiguiente *reward* luego de la acción $A_t$ que será $R_{t+1} \in \{-1,0,1\}$.

Con estos valores podemos volver a: 

>    critic_loss = critic_loss + ((Gt - value)\*\*2) / 2

Y ver que **el valor máximo** que puede tomar será $0 \leq \frac{1}{2}(R - \text{value})^2 \leq 2$ , pues la diferencia máxima entre $R$ y $\text{value}$ es $2$

* Clamp on loss 
![loss-before-clamp.png](./img/loss-before-clamp.png)

* Clamp on Critic value
![loss-after-clamp.png](./img/loss-after-clamp.png)

In [ ]:
import os
os.chdir('/home/jack/TP-Final-Procesos-Markovianos-para-el-Aprendizaje-Automatico-2019-1C/gym_pacman')

from train import train
from argparse import Namespace
args = Namespace(beta =0.001,
                 gamma=0.9,
                 tau=1.0,
                 lr=1e-4,
                 layout='atari',
                 load_previous_weights=True,
                 num_processes=6,                # numero de procesos asincronicos
                 num_processes_to_render=1,     # numero de procesos a visualizar
                 use_gpu=False,
                 max_actions=200,
                 num_global_steps=5e9,
                 num_local_steps=50,     # async updates every
                 save_interval=25,
                 saved_path='trained_models',
                 log_path='tensorboard',
                 record=False)
print("Agent progress details are displayed on this jupyter notebook TERMINAL (not here)\n")
train(args)

Agent progress details are displayed on this jupyter notebook TERMINAL (not here)

Getting number of NN inputs/outputs for atari
Loading previous weights for atari... Done.


In [1]:
max(1,2,3)

3